# Schwachstellenanalyse hinsichtlich bekannter CVE's

## Fragestellung

Treten in dem vorliegendem Java-Projekt mögliche Sicherheitslücken aufgrund von gemeldeten CVE auf? Wie kritisch werden jene Schwachstellen eingeschätzt?
* relevant für Dienstleister/Entwickler und Kunden/Anwender zugleich um Angriffe von Dritten über die Ausnutzung von Schwachstellen zu unterbinden (z.B. Sniffer-Attacks, Denial-of-Service, Buffer-Overflow)
* Ergebnisse sollen weiterführende Analysemöglichkeiten und Risikobewertuntgen ermöglichen

## Datenquelle
* Java-Strukturen eines Git-Projekts, welches aufgrund der Anpassung der pom.xml durch jQAssistant gescannt und in einer Neo4j-Instanz gespeichert wurde
* Daten zu CVE über Dateiimport und Request an cveapi (letzteres nur für einen übersichtlichen Datensatz möglich, da die Firewall des API-Servers den Request zum Schutz vor Denial-of-Service-Attacken sperrt)

### konkrete Quellen
* Projektgrundlage: Petclinic &rarr; verwendete Frameworks (Artefakte) & zugehörige Versionen
* CVE-Dateiimport: historische CVE von 2021 als JSON-Datei (Stand: 23.08.2021) <br>
&rarr; Download älterer Dateien unter https://nvd.nist.gov/vuln/data-feeds möglich
* CVE API über NVD: Import der 20 aktuellsten CVE oder aller CVE über einen definierten Erstellungszeitraum <br>
&rarr; Anpassung über URL möglich <br>
&rarr; Höchstens 20 CVE sind in der Response enthalten.

### mögliche Schwachstellen
* Artefakte/Frameworks ohne Versionsnummer, Versionsnummer mit unbrauchbaren Präfix/Suffix
* Artefakte/Frameworks, welche nicht über jQAssistant erfasst werden konnten
* genaues Matching zwischen verwendete Version und schwachstellenrelevante Version

## Annahmen
* Die relevanten Artefakte konnten von jQAssistant gescannt und als Artefakte mit den entsprechenden Labels "name" und "version" gespeichert werden. 
* Relevante Daten wie betroffene Konfigurationen, Versionen und Schweregrad der Schwachstellen können über die JSON-Strukturierung der CVE abgerufen werden.
* Die Artefakte können mit den Informationen der CVE abgeglichen werden. 

## Validierung

### Datenaufbereitung
* Tabellenansicht bzgl. aller relevanten (im Projekt möglichen) Schwachstellen
* Graphische Übersicht über betroffene Artefakte und Anzahl der zugehörigen CVE
* Graphische Übersicht über hohe (zu analysierende) Impacts der auftretenden CVE bzgl. einer Auswahl von Artefakten
* Graphische Übersicht über Base Score der auftretenden CVE


### Aktionen
* Review bzgl. der auftretenden Schwachstellen und dessen Schweregrade
* Validierung & Evaluierung der Schwachstellen durch Domainexperten
* Planung der nächsten Schritte/Meilensteine durch Projekt

## Implementierung
* Identifikation der relevanten Artefakte über Node Label "Artifact" und über die Property-Nodes des Nodes der pom.xml mit der Relationship [:HAS_PROPERTY] &rarr; Extraktion des Namens & der Version
* Extraktion der CVE aus der JSON-Datei & über cveapi &rarr; Auflösung von Verschachtelungen und Kürzung der Spaltenanzahl
* Abgleich der identifizierten Artefakte aus dem Projekt mit den betroffenden Konfiguration aus den CVE

In [1]:
#Import of all used libraries
import py2neo
import pandas as pd
import numpy as np

import json
from pandas.io.json import json_normalize
import openpyxl

import urllib3
from urllib3 import request
import json
import certifi

from IPython.display import display, HTML
import pygal

In [2]:
base_html = """
<!DOCTYPE html>
<html>
  <head>
  <script type="text/javascript" src="http://kozea.github.com/pygal.js/javascripts/svg.jquery.js"></script>
  <script type="text/javascript" src="https://kozea.github.io/pygal.js/2.0.x/pygal-tooltips.min.js""></script>
  </head>
  <body>
    <figure>
      {rendered_chart}
    </figure>
  </body>
</html>
"""

In [3]:
#Verbindung zu neo4j und Speicherung des Graphen in der Variable 'Graph'

graph = py2neo.Graph(host='localhost', user='neo4j', password='neo4j')

In [4]:
#Query um alle Artefakte bzgl. Frameworks zu erhalten
#Bereininung von Duplikaten, Testfiles und unnötigen Präfixes 

query = """
MATCH (artifact:Artifact) WHERE NOT artifact.name contains 'petclinic' AND NOT artifact.type = 'test-jar'
WITH DISTINCT artifact 
Return artifact.name as Artefakt, artifact.version as Version
"""
df_usedArtifacts = pd.DataFrame(graph.run(query), columns=['Artefakt', 'Version'])
df_usedArtifacts['Version'] = df_usedArtifacts['Version'].str.replace('[.-]?[a-zA-Z]+[-]?\w+([.-]?\d*)*$','',regex=True)

In [5]:
#Query zu allen Frameworks, die als Property an der pom.xml gespeichert sind
#Joining beider Dataframes, Duplikaterntfernung

query2 = """
Match (p:Pom)-[:HAS_PROPERTY]->(pr:Property) 
Return pr.name as Artefakt, pr.value as Version
"""
df_PomProperties = pd.DataFrame(graph.run(query2), columns=['Artefakt', 'Version'])
df_PomProperties['Artefakt'] = df_PomProperties['Artefakt'].str.replace('.?[vV]ersion','',regex=True)

df_usedItems = pd.merge(left=df_PomProperties, right=df_usedArtifacts, how='outer', left_on='Artefakt', right_on='Artefakt')
for i in df_usedItems.index:
    version_y = df_usedItems['Version_y'][i]
    version_x = df_usedItems['Version_x'][i]
    if pd.isnull(version_x) and (version_y is not None):
        df_usedItems.loc[i, 'Version_x'] = version_y

df_usedItems = df_usedItems.sort_values(by=['Artefakt','Version_x'], ascending=False, na_position='last')
df_usedItems = df_usedItems.drop_duplicates(subset='Artefakt', keep="first")
df_usedItems = df_usedItems.reset_index()
df_usedItems = df_usedItems.drop(['index'], axis=1)

df_usedItems = df_usedItems.drop(['Version_y'], axis=1)
df_usedItems.columns =['Artefakt', 'Version']
df_usedItems

,Artefakt,Version
0,xmlunit2,2.6.2
1,xmlunit-matchers,2.6.2
2,xmlunit-legacy,2.6.2
3,xmlunit-core,2.6.2
4,xml-path,3.1.1
...,...,...
982,activemq,5.15.7
983,accessors-smart,1.2
984,LatencyUtils,2.0.3
985,HikariCP,3.2.0


## Datenimport über CVE API oder entsprechender JSON
Beide Möglichkeiten besitzen eine ähnliche Datenstrukturierung. Der Request über cveapi erweitert die Strukturierung nur um wenige weitere Key-Value-Paare, weshalb das DataFrame zusätzlicher Anpassung benötigt.

In [6]:
# Laden der statischen CVE-Daten über JSON-Datei
# Download über https://nvd.nist.gov/vuln/data-feeds

api = 'false'

with open('CVE/nvdcve-1.1-2021.json', encoding='utf-8') as staticData:
    jsonData = json.load(staticData)
df_raw = pd.json_normalize(jsonData, record_path =['CVE_Items'])


In [ ]:
# Datenimport über cveapi 
# Import aller CVE's wird nicht empfohlen, da dies von der Firewall des Servers
# zur Prävention von Denial-of-service-Attacken verhindert wird
# Response enthält immer höchstens 20 CVE

api = 'true'

http = urllib3.PoolManager(
    cert_reqs="CERT_REQUIRED",
    ca_certs=certifi.where()
)

#Request der 20 aktuellsten CVE
url ='https://services.nvd.nist.gov/rest/json/cves/1.0?startIndex=20' 

#Request der CVE ab einem definierten Startzeitpunkt
#url ='https://services.nvd.nist.gov/rest/json/cves/1.0?pubStartDate=2021-08-01T00:00:00:000 UTC-05:00'


r = http.request('GET', url)
r.status

# JSON-Daten werden ausgewertet & in ein Dictionary gespeichert
jsonData = json.loads(r.data.decode('utf-8'))
df_nested_list = pd.json_normalize(jsonData)
df_raw = df_nested_list.loc[:,df_nested_list.columns.isin(['result.CVE_Items'])]
json_struct = json.loads(df_raw.to_json(orient="records")) 
df_raw = pd.json_normalize(json_struct,record_path =['result.CVE_Items'])

## Datenaufbereitung über DataFrames
Daten aus dem Request oder der JSON-Datei werden in mehreren DataFrames entschachtelt und aufbereitet, um für die darauffolgende Filterung bzgl. der verwendeten Artefakte vorbereitet zu werden.

In [7]:
#Aufbereitung der Daten zu einer Tabelle mit ID, Beschreibung & Schweregrad der Sicherheitslücke

#DataFrame wird auf 6 Spalten gekürzt & Spalten werden umbenannt (Lesbarkeit)
vulnerableList= df_raw.columns.isin(['cve.CVE_data_meta.ID', 'impact.baseMetricV3.cvssV3.confidentialityImpact', 'impact.baseMetricV3.cvssV3.integrityImpact', 'impact.baseMetricV3.cvssV3.availabilityImpact', 'impact.baseMetricV3.cvssV3.baseScore', 'impact.baseMetricV3.exploitabilityScore', 'impact.baseMetricV3.impactScore'])
df_basic = df_raw.loc[:,vulnerableList]
df_basic.columns =['CVE-ID', 'Confidentially Impact', 'Integrity Impact', 'Availability Impact','Base Score', 'Exploitability Score', 'Impact Score']


#Neues DF mit den CVE-Beschreibungen, da "cve.description.description_data" ein Dictionary enthält
df_raw2 = df_raw.loc[:,df_raw.columns.isin(['cve.CVE_data_meta.ID', 'cve.description.description_data'])]

#Reload & Manipulation des DataFrames um an die entsprechende Beschreibung zu gelangen
json_struct = json.loads(df_raw2.to_json(orient="records")) 
df_desc = pd.json_normalize(json_struct,record_path =['cve.description.description_data'], meta=['cve.CVE_data_meta.ID'])
df_desc = df_desc.loc[:,df_desc.columns.isin(['value', 'cve.CVE_data_meta.ID'])]
df_desc.columns =['CVE-Beschreibung', 'CVE-ID']

#DF-Join von df_basic & df_desc
basicList = pd.merge(left=df_basic, right=df_desc, left_on='CVE-ID', right_on='CVE-ID')
basicList = basicList[['CVE-ID', 'CVE-Beschreibung', 'Confidentially Impact', 'Integrity Impact', 'Availability Impact','Base Score', 'Impact Score', 'Exploitability Score']]

In [8]:
#Neues DF mit den Konfigurationsbeschreibungen, da "configurations.nodes" ein Dictionary enthält
newList= df_raw.columns.isin(['cve.CVE_data_meta.ID', 'configurations.nodes'])
df_raw3 = df_raw.loc[:,newList]

#Reload & Manipulation des DataFrames um an die entsprechende fehlerhafte Konfiguration zu gelangen
json_struct = json.loads(df_raw3.to_json(orient="records")) 
df_conf = pd.json_normalize(json_struct,record_path =['configurations.nodes'], meta=['cve.CVE_data_meta.ID'])
json_struct = json.loads(df_conf.to_json(orient="records")) 
df_conf2 = pd.json_normalize(json_struct,record_path =['cpe_match'], meta=['operator', 'cve.CVE_data_meta.ID'])

#Kürzung & Umbenennung der Spalten
if api == 'false':
    df_conf2 = df_conf2.loc[:,df_conf2.columns.isin(['cpe23Uri', 'versionEndIncluding', 'versionEndExcluding', 'versionStartIncluding', 'versionStartExcluding', 'operator', 'cve.CVE_data_meta.ID'])]
    df_conf2.columns =['cpe23URI', 'Last version (excl)', 'First version (incl)', 'Last version (incl)', 'First version (excl)', 'Connector/Relation', 'CVE-ID']
    df_conf2 = df_conf2[['CVE-ID', 'Connector/Relation', 'cpe23URI','First version (excl)', 'First version (incl)', 'Last version (excl)', 'Last version (incl)']]
elif api == 'true':
    df_conf2 = df_conf2.rename(columns={"cve.CVE_data_meta.ID": "CVE-ID", "cpe23Uri": "cpe23URI", "operator": "Connector/Relation"})
    df_conf2 = df_conf2.drop(columns=['cpe_name'])
 

In [9]:
#DataFrame bzgl. Konfigurationen mit Schwachstellen mit den verwendeten Artefakten scannen
list1 = []
list2 = []
df_result = df_conf2[0:0]
df_result.insert(len(df_result.columns), "verwendetes Artefakt", [])
df_result.insert(len(df_result.columns), "verwendete Version", [])

for j in df_usedItems.index:
    version = df_usedItems['Version'][j]
    artefakt = df_usedItems['Artefakt'][j]
    
    df_search = df_conf2.loc[df_conf2['cpe23URI'].str.contains(':'+artefakt + ':', case=False)]
    if df_search is not None:
        lengthDF = df_search.shape[0]
        for i in range(lengthDF):
            list1.append(artefakt)
            list2.append(version)
        df_search.insert(len(df_search.columns), "verwendetes Artefakt", list1)
        df_search.insert(len(df_search.columns), "verwendete Version", list2)
        list1.clear()
        list2.clear()
    df_result = df_result.append(df_search, ignore_index=True)

### Auflistung der Schwachstellen
Die folgende Tabelle stellt alle möglichen Schwachstellen bzgl. der verwendeten Artefakte in dem gescannten Projekt dar. Es werden neben der CVE-ID & dem im Projekt verwendeten Artefakt + Versionsnummer auch eine CVE-Beschreibung und die jeweiligen Scores zur Einschätzung der Relevanz und des Schweregrades aufgelistet.
Die Liste wird als Excel abgespeichert.

In [10]:
#Neues DataFrame mit kompakten Daten (ohne Versionenvergleich)
df_compact = df_result[0:0]
df_compact = df_result.drop_duplicates(subset='CVE-ID', keep="first")
df_compact = df_compact.loc[:,df_compact.columns.isin(['CVE-ID', 'verwendetes Artefakt', 'verwendete Version'])]
df_compact = pd.merge(left=basicList, right=df_compact, left_on='CVE-ID', right_on='CVE-ID')
df_compact.to_excel("result_analysis.xlsx")
df_compact

,CVE-ID,CVE-Beschreibung,Confidentially Impact,Integrity Impact,Availability Impact,Base Score,Impact Score,Exploitability Score,verwendetes Artefakt,verwendete Version
0,CVE-2021-1998,Vulnerability in the MySQL Server product of O...,NONE,LOW,LOW,3.8,2.5,1.2,mysql,8.0.13
1,CVE-2021-2001,Vulnerability in the MySQL Server product of O...,NONE,NONE,HIGH,4.9,3.6,1.2,mysql,8.0.13
2,CVE-2021-2002,Vulnerability in the MySQL Server product of O...,NONE,NONE,HIGH,4.9,3.6,1.2,mysql,8.0.13
3,CVE-2021-2006,Vulnerability in the MySQL Client product of O...,NONE,NONE,HIGH,5.3,3.6,1.6,mysql,8.0.13
4,CVE-2021-2007,Vulnerability in the MySQL Client product of O...,LOW,NONE,NONE,3.7,1.4,2.2,mysql,8.0.13
...,...,...,...,...,...,...,...,...,...,...
125,CVE-2021-3393,An information leak was discovered in postgres...,LOW,NONE,NONE,4.3,1.4,2.8,postgresql,42.2.5
126,CVE-2021-34428,"For Eclipse Jetty versions <= 9.4.40, <= 10.0....",LOW,LOW,NONE,3.5,2.5,0.9,jetty,9.4.12.v20180830
127,CVE-2021-34429,"For Eclipse Jetty versions 9.4.37-9.4.42, 10.0...",LOW,NONE,NONE,5.3,1.4,3.9,jetty,9.4.12.v20180830
128,CVE-2021-34629,The SendGrid WordPress plugin is vulnerable to...,LOW,NONE,NONE,4.3,1.4,2.8,sendgrid,4.3.0


#### Legende
*Base Score* <br>
= Repräsentation des natürlichen Charakters und des Schweregrads einer Schwachstelle (konstant über einen längeren Zeitraum, über verschiedene Umgebungen hinweg) <br>
&rarr; beinhaltet den Impact Score & den Exploitability Score

*Exploitability Score* <br>
= Reflektion der angreifbaren Komponente &rarr; beinhaltet unteranderem die Situation und den Kontext, der den Angriff ermöglichen kann

*Impact Score* <br>
= Schweregrad der direkten Konsequenzen eines erfolgreichen Exploits auf den betroffenen "Gegenstand" (Softwaresystem, Umgebung, Daten, ...) und der direkte & vorhersehbare Effekt der ausgenutzten Schwachstelle 


**Mögliche Ausprägungen**<br>
*None*: 0 <br>
*Low*: 0.1 - 3.9<br>
*Medium*: 4.0 - 6.9<br>
*High*: 7.0 - 8.9<br>
*Severe*: 9.0 - 10.0

### Visualisierung
Im folgenden Abschnitt wird mithilfe einiger Charts der Zusammenhang zwischen CVE, betroffenes Artefakt, Auftrittshäufigkeit und Schweregrade dargestellt.

In [11]:
df_count = df_compact['verwendetes Artefakt'].value_counts().to_frame()
df_count['Artefakt'] = df_count.index
df_count.columns =['Anzahl auftretender CVE', 'Artefakt']
df_count.columns.name = None
df_count = df_count.reset_index()
df_count = df_count.loc[:,df_count.columns.isin(['Anzahl auftretender CVE', 'Artefakt'])]
df_count

,Anzahl auftretender CVE,Artefakt
0,84,mysql
1,9,elasticsearch
2,8,mongodb
3,6,tomcat
4,6,jetty
5,5,postgresql
6,3,netty
7,3,solr
8,1,jackson-databind
9,1,undertow


In [12]:
pie_chart = pygal.Pie()
pie_chart.title = 'Anzahl der Artefakte mit Sicherheitsbedenken'
for i in range(len(df_count)):
    artefakt= df_count['Artefakt'][i]
    anzahl=df_count['Anzahl auftretender CVE'][i]
    pie_chart.add(artefakt, anzahl)
display(HTML(base_html.format(rendered_chart=pie_chart.render(is_unicode=True))))

### Verteilung der hohen Impactbewertungen über die zehn häufigsten Artefakte

**Legende** <br>
* *Confidentiality Impact* = Einfluss auf die Vertraulichkeit der Informationsgewinnung durch das System (Beinhaltet z.B. begrenzter Informationszugriff, Zugriff nur per Authorisierung)
* *Integrity Impact* = Einfluss auf den Wahrheitswert der zu schützenden Informationen durch z.B. unautorisierte & unbemerkte Manipulation durch Angreifer
* *Availability Impact* = Einfluss auf die Verfügbarkeit der anzugreifenden Komponente durch einen Exploit


In [13]:
from pygal.style import DefaultStyle
top5 = df_count.head(10)
labels = []
impactTypes = ['Confidentially Impact', 'Integrity Impact', 'Availability Impact']
count_ci = []
count_ii = []
count_ai = []

line_chart = pygal.StackedBar(show_legend=True, human_readable=True, fill=True, legend_at_bottom=True, print_values=True, style=DefaultStyle(value_font_size=12))

line_chart.title = 'Verteilung der hohen Impacts unter den Top 10'
line_chart.x_title= 'betroffene Artefakte'
line_chart.y_title='Anzahl der Impact-Treffer'

for j in top5.index:
    artefakt = top5['Artefakt'][j]
    
    
    df_interimResult = df_compact.loc[df_compact['verwendetes Artefakt'].str.contains(artefakt, case=False)]
    
    count_hi = len(df_interimResult[(df_interimResult[impactTypes[0]] == 'HIGH') | (df_interimResult[impactTypes[0]] == 'HIGH') | (df_interimResult[impactTypes[2]] == 'HIGH')])
    
    count_ci.append(len(df_interimResult[df_interimResult[impactTypes[0]] == 'HIGH']))
    count_ii.append(len(df_interimResult[df_interimResult[impactTypes[1]] == 'HIGH']))
    count_ai.append(len(df_interimResult[df_interimResult[impactTypes[2]] == 'HIGH']))
    labels.append(artefakt+' ('+ str(count_hi)+')')

line_chart.add(impactTypes[0], count_ci)
line_chart.add(impactTypes[1], count_ii)
line_chart.add(impactTypes[2], count_ai)

line_chart.x_labels = labels
    

display(HTML(base_html.format(rendered_chart=line_chart.render(is_unicode=True))))


In [14]:
df_interimResult = df_compact[0:0]
cveListC = []
cveListH = []
cveListM = []
cveListL = []
cveListN = []


treemap_BaseS = pygal.Treemap()
treemap_BaseS.title = 'Schweregrad der jeweiligen Schwachstelle (CVE) anhand des Base Score'

for j in df_count.index:
    artefakt = df_count['Artefakt'][j]
    df_interimResult = df_compact.loc[df_compact['verwendetes Artefakt'].str.contains(artefakt, case=False)]
    
    for i in df_interimResult.index:
        baseScore = df_interimResult['Base Score'][i]
        if baseScore >= 9:
            cveListC.append({'value': df_interimResult['Base Score'][i], 'label': (df_interimResult['CVE-ID'][i]+': ' + artefakt)})
        elif baseScore >= 7 and baseScore < 9:
            cveListH.append({'value': df_interimResult['Base Score'][i], 'label': (df_interimResult['CVE-ID'][i]+': ' + artefakt)})
        elif baseScore >= 4 and baseScore < 7:
            cveListM.append({'value': df_interimResult['Base Score'][i], 'label': (df_interimResult['CVE-ID'][i]+': ' + artefakt)})
        elif baseScore < 4:
            cveListL.append({'value': df_interimResult['Base Score'][i], 'label': (df_interimResult['CVE-ID'][i]+': ' + artefakt)})
        elif baseScore == 0:
            cveListN.append({'value': df_interimResult['Base Score'][i], 'label': (df_interimResult['CVE-ID'][i]+': ' + artefakt)})
    
treemap_BaseS.add('Critical', cveListC)
treemap_BaseS.add('High', cveListH)
treemap_BaseS.add('Medium', cveListM)
treemap_BaseS.add('Low', cveListL)
treemap_BaseS.add('None', cveListN)
    
 

display(HTML(base_html.format(rendered_chart=treemap_BaseS.render(is_unicode=True))))

CVE mit den höchsten Werten sollten zuerst gegen geprüft werden. &rarr; mögliche Priorisierung

## Ergebnisse

### Softwareanalyse
* Die häufigsten CVE bzgl. Petclinic treten aufgrund der Verwendung von mysql. (2020 unteranderem auch FasterXML jackson-databind)
* Besonders häufig werden Schwachstellen bzgl. der Verfügbarkeit ausgenutzt.
* Anhand des Base Scores konnten 2 Artefakte (postgresql & solr) mit je kritischen 2 CVE identifiziert werden, deren Schwachstellen möglichst zeitnah analysiert werden sollten.

### Hindernisse/Verbesserungspotentiale
* betroffene Artefaktversionen innerhalb der JSON-Datei/Response werden in vier statt zwei Spalten gepflegt <br>
&rarr; Verhinderung von weiterer Eingrenzung der Liste<br>
&rarr; zusätzliche manuelle Einkürzung notwendig
* fehlende Versionsnummern an einigen Artefakten
* Verbesserung der Schnittstelle zur API
* Verbesserung der Regex-Ausdrücke
* Interessante Idee: Relation zwischen betroffene Artefakte, Klassen und Codezeilen

## Nächste Schritte
* Präsentation der Ergebnisse und Diskussion mit den Domainexperten und dem Projekt (womöglich einem Teil der Stakeholder)
* Sichtung der Ergebnistabelle und weitere Kürzung der CVE-Liste
* Planung & Aufwandsschätzung von möglichen Updates & Bugfixes
* Regelmäßige CVE-Scans planen

## Quellen

- Harrer, M., Software Analytics Canvas, URL: https://www.feststelltaste.de/software-analytics-canvas/, gelesen am 24.07.2021
- o.V., NVD Data Feeds, URL: https://nvd.nist.gov/vuln/data-feeds, gelesen am 31.07.2021
- o.V., Vulnerability Metrics, URL: https://nvd.nist.gov/vuln-metrics/cvss, gelesen am 26.08.2021
- o.V., Common Vulnerability Scoring System version 3.1: Specification Document, URL: https://www.first.org/cvss/specification-document, gelesen am 26.08.2021